In [2]:
import csv
import tensorflow as tf
import itertools
import operator
import numpy as np
import nltk
import sys
from datetime import datetime

In [56]:
vocabulary_size = 8000
unknown_token = "UNKNOWN_TOKEN"

with open('groceries_mba.csv', 'r',encoding='utf-8') as f:
    reader = csv.reader(f)
    basket= [x for x in reader]

In [57]:
basket[0], len(basket)

(['citrus_fruit', 'semi-finished_bread', 'margarine', 'ready_soups'], 9835)

In [49]:
basket[:10]

[['citrus_fruit', 'semi-finished_bread', 'margarine', 'ready_soups'],
 ['tropical_fruit', 'yogurt', 'coffee'],
 ['whole_milk'],
 ['pip_fruit', 'yogurt', 'cream_cheese_', 'meat_spreads'],
 ['other_vegetables',
  'whole_milk',
  'condensed_milk',
  'long_life_bakery_product'],
 ['whole_milk', 'butter', 'yogurt', 'rice', 'abrasive_cleaner'],
 ['rolls/buns'],
 ['other_vegetables',
  'UHT-milk',
  'rolls/buns',
  'bottled_beer',
  'liquor_(appetizer)'],
 ['pot_plants'],
 ['whole_milk', 'cereals']]

In [24]:
import pickle
with open('data.txt', 'wb') as f:
    pickle.dump(basket, f)

In [25]:
import pickle
with open ('words.txt', 'rb') as fp:
    wordlist = pickle.load(fp)

In [26]:
with open('embeddings.txt','rb') as fp:
    embeddings = pickle.load(fp)

In [31]:
wordlist.append('UNKNOWN_TOKEN')

In [32]:
embeddings = np.vstack([embeddings, np.zeros(200)])

In [34]:
len(wordlist), len(embeddings)

(170, 170)

In [35]:
word_to_index = dict([(w,i) for i,w in enumerate(wordlist)])

In [36]:
word_to_index

{'Instant_food_products': 99,
 'UHT-milk': 38,
 'UNKNOWN_TOKEN': 169,
 'abrasive_cleaner': 128,
 'artif._sweetener': 131,
 'baby_cosmetics': 163,
 'baby_food': 167,
 'bags': 164,
 'baking_powder': 63,
 'bathroom_cleaner': 139,
 'beef': 26,
 'berries': 40,
 'beverages': 50,
 'bottled_beer': 12,
 'bottled_water': 5,
 'brandy': 124,
 'brown_bread': 18,
 'butter': 24,
 'butter_milk': 48,
 'cake_bar': 74,
 'candles': 93,
 'candy': 44,
 'canned_beer': 14,
 'canned_fish': 72,
 'canned_fruit': 133,
 'canned_vegetables': 81,
 'cat_food': 56,
 'cereals': 111,
 'chewing_gum': 58,
 'chicken': 30,
 'chocolate': 28,
 'chocolate_marshmallow': 91,
 'citrus_fruit': 11,
 'cleaner': 118,
 'cling_film/bags': 78,
 'cocoa_drinks': 144,
 'coffee': 22,
 'condensed_milk': 86,
 'cooking_chocolate': 141,
 'cookware': 138,
 'cream': 153,
 'cream_cheese_': 32,
 'curd': 25,
 'curd_cheese': 119,
 'decalcifier': 152,
 'dental_care': 110,
 'dessert': 36,
 'detergent': 59,
 'dish_cleaner': 84,
 'dishes': 65,
 'dog_food

In [37]:
word_to_index['UNKNOWN_TOKEN']

169

In [39]:
for i, sent in enumerate(basket):
    basket[i] = [w if w in word_to_index else unknown_token for w in sent]

In [199]:
import numpy as np
X_train = np.asarray([[word_to_index[w] for w in sent[:-1]] for sent in basket])
y_train = np.asarray([[word_to_index[w] for w in sent[1:]] for sent in basket])

In [200]:
x_example, y_example = X_train[10], y_train[10]
print("x:\n%s\n%s" % (" ".join([wordlist[x] for x in x_example]), x_example))
print("\ny:\n%s\n%s" % (" ".join([wordlist[x] for x in y_example]), y_example))

x:
tropical_fruit other_vegetables white_bread bottled_water
[7, 1, 31, 5]

y:
other_vegetables white_bread bottled_water chocolate
[1, 31, 5, 28]


In [201]:
len(X_train), X_train.shape

(9835, (9835,))

In [202]:
X_train[:10]

array([[11, 64, 21], [7, 4], [], [15, 4, 32], [1, 0, 86], [0, 24, 4, 101],
       [], [1, 38, 2, 12], [], [0]], dtype=object)

In [203]:
y_train[:10]

array([[64, 21, 148], [4, 22], [], [4, 32, 122], [0, 86, 35],
       [24, 4, 101, 128], [], [38, 2, 12, 100], [], [111]], dtype=object)

In [204]:
X_train = np.array([var for var in X_train if var])
y_train = np.array([var for var in y_train if var])

In [205]:
len(X_train), len(y_train)

(7676, 7676)

In [206]:
X_train.shape

(7676,)

In [207]:
basket[10]

['tropical_fruit',
 'other_vegetables',
 'white_bread',
 'bottled_water',
 'chocolate']

In [208]:
np.average([len(row) for row in X_train])

4.3684210526315788

In [209]:
a = [len(row) for row in X_train]
a.index(max(a))

959

In [226]:
import tensorflow as tf
import numpy as np
x = X_train
y = y_train
max_length =  10
x_padded = []
y_padded = []


for row in x:
    if len(row) <= max_length:
        x_padded.append(row + [169] * (max_length - len(row)))
    else :
        x_padded.append(row[:10])

for row in y:
    if len(row) <= max_length:
        y_padded.append(row + [169] * (max_length - len(row))) 
    else: 
        y_padded.append(row[:10])

x_padded = np.array(x_padded)
y_padded = np.array(y_padded)

In [229]:
x_padded.shape, y_padded.shape

((7676, 10), (7676, 10))

In [230]:
y_padded

array([[ 64,  21, 148, ..., 169, 169, 169],
       [  4,  22, 169, ..., 169, 169, 169],
       [  4,  32, 122, ..., 169, 169, 169],
       ..., 
       [ 11,   1,  24, ..., 121,  78, 169],
       [  5,   3,  12, ..., 169, 169, 169],
       [  7,   1, 107, ..., 169, 169, 169]])

In [231]:
len(embeddings)

170

In [232]:
len(x_padded),len(y_padded)

(7676, 7676)

In [233]:
xtrain, ytrain = x_padded[:7000], y_padded[:7000]
xtest, ytest = x_padded[7000:7600], y_padded[7000:7600]

In [246]:
tf.reset_default_graph()
from tensorflow.contrib import rnn 
hm_epochs = 150
batch_size = 500
num_steps = 10
state_size = 200
vocab_size = 170

# Placeholders
x = tf.placeholder(tf.int32, [None, num_steps]) # [batch_size, num_steps]
seqlen = tf.placeholder(tf.int32, [None])
y = tf.placeholder(tf.int32, [None, num_steps])

def train_neural_network(x): 
    
        def next_batch(step):
            p,q = xtrain[batch_size*step:batch_size*(step+1)], ytrain[batch_size*step:batch_size*(step+1)]
            return p,q
        
        def test_batch(stp):    
            a,b = xtest[batch_size*stp:batch_size*(stp+1)], ytest[batch_size*stp:batch_size*(stp+1)]
            return a,b
              
        def lstm_neural_network(x):
            # Embedding layer
            rnn_inputs = tf.nn.embedding_lookup(embeddings, x)
            rnn_inputs = tf.cast(rnn_inputs, dtype=tf.float32)
            print(rnn_inputs)
            
            # RNN
            inputs = tf.unstack(rnn_inputs, num=num_steps, axis=1)   
            cell = tf.contrib.rnn.GRUCell(state_size)
            cell = tf.contrib.rnn.DropoutWrapper(cell=cell, output_keep_prob = 0.5)
            print("this is rnn going in:", rnn_inputs)
            rnn_outputs, rnn_states = tf.contrib.rnn.static_rnn(cell, inputs, dtype=tf.float32)

            rnn_output = tf.reshape(tf.concat(axis=1, values=rnn_outputs), [-1, state_size])
            print("this is the output:",rnn_output)
            softmax_w = tf.get_variable("softmax_w", [state_size, vocab_size], dtype=tf.float32)
            softmax_b = tf.get_variable("softmax_b", [vocab_size], dtype=tf.float32)
            logits = tf.add(tf.matmul(rnn_output, softmax_w),softmax_b, name = "logits")
            return logits

        prediction = lstm_neural_network(x)
        loss = tf.contrib.legacy_seq2seq.sequence_loss_by_example(
            [prediction],
            [tf.reshape(y[:batch_size], [-1])],
            [tf.ones([batch_size * num_steps], dtype=tf.float32)])
        cost = tf.reduce_sum(loss) / batch_size
        optimizer = tf.train.AdamOptimizer().minimize(cost)   
        saver = tf.train.Saver()
        
        with tf.Session() as sess:
            sess.run(tf.initialize_all_variables())
            for epoch in range(hm_epochs):
                epoch_loss = 0 
                for step in range(int(len(xtrain)/batch_size)):
                    epoch_x, epoch_y = next_batch(step)
                    _, c = sess.run([optimizer, cost], feed_dict={x: epoch_x, y: epoch_y})
                    epoch_loss += c
                print('Epoch', epoch, 'completed out of',hm_epochs,'loss:',epoch_loss)
            save_path = saver.save(sess, "./model")
            print("Model saved in file: %s" % save_path)
            
            for stp in range(int(len(xtest)/batch_size)):
                s,u = test_batch(stp)
                correct = tf.equal((tf.argmax(prediction,1)),tf.cast(tf.reshape(tf.concat(axis=1, values= u), [-1]),tf.int64))
                accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
                print( 'Batch #', stp, 'Accuracy by each batch:',accuracy.eval({x: s, y: u}))

In [247]:
train_neural_network(x)

Tensor("Cast:0", shape=(?, 10, 200), dtype=float32)
this is rnn going in: Tensor("Cast:0", shape=(?, 10, 200), dtype=float32)
this is the output: Tensor("Reshape:0", shape=(?, 200), dtype=float32)
Instructions for updating:
Use `tf.global_variables_initializer` instead.
Epoch 0 completed out of 150 loss: 566.010211945
Epoch 1 completed out of 150 loss: 355.764188766
Epoch 2 completed out of 150 loss: 307.057319641
Epoch 3 completed out of 150 loss: 280.411174774
Epoch 4 completed out of 150 loss: 261.422828674
Epoch 5 completed out of 150 loss: 246.748120308
Epoch 6 completed out of 150 loss: 235.012883186
Epoch 7 completed out of 150 loss: 226.932684898
Epoch 8 completed out of 150 loss: 220.55629921
Epoch 9 completed out of 150 loss: 215.752729416
Epoch 10 completed out of 150 loss: 211.853147507
Epoch 11 completed out of 150 loss: 208.677114487
Epoch 12 completed out of 150 loss: 206.18270874
Epoch 13 completed out of 150 loss: 204.025616646
Epoch 14 completed out of 150 loss: 202.3

In [271]:

sess=tf.Session()    
#First let's load meta graph and restore weights
saver = tf.train.import_meta_graph('model.meta')
saver.restore(sess,tf.train.latest_checkpoint('./'))
sess.run(tf.global_variables_initializer())
graph = tf.get_default_graph()

#context = graph.get_tensor_by_name("context:0") 
#utterance = graph.get_tensor_by_name("utterance:0")
#target = graph.get_tensor_by_name("target:0")

op_to_restore = graph.get_tensor_by_name("logits:0")
#qe,an,lbl = test_batch(0)
feed_dict={x: x_padded[7606:7676]}
results = sess.run(op_to_restore,feed_dict)
print("These are results:", results)
results = np.reshape(results, [70, num_steps, vocab_size])

These are results: [[-0.06714499  0.03206809  0.50385636 ...,  0.0839908   0.28306127
   0.07631007]
 [-0.06654603 -0.15623781  0.32606471 ...,  0.44879982 -0.25671446
  -0.10963421]
 [-0.51463908 -0.35265481  0.40559691 ...,  1.10341454 -0.22839133
  -0.40540844]
 ..., 
 [-0.13943213  0.06104822  0.24092276 ..., -0.12281565  0.05632129
   0.17111006]
 [-0.19139811  0.00268919  0.31316239 ...,  0.00463292  0.12506922
   0.04600501]
 [-0.02362924 -0.0138546   0.22703174 ..., -0.04348115  0.13229501
   0.23531948]]


In [275]:
r1= results.argmax(axis=2)[:10]

In [276]:
r2=y_padded[7606:7676][:10]

In [297]:
for pre,the in zip(r1,r2):
    print("\nx: These are the predictions made \n%s" % (",".join([wordlist[x] for x in pre])))
    print("\ny These are the actual present in dataset:\n%s" % (",".join([wordlist[x] for x in the])))


x: These are the predictions made 
seasonal_products,toilet_cleaner,baby_food,abrasive_cleaner,pot_plants,flour,liver_loaf,flour,flour,canned_vegetables

y These are the actual present in dataset:
whole_milk,whipped/sour_cream,sliced_cheese,domestic_eggs,rolls/buns,margarine,bottled_water,fruit/vegetable_juice,UNKNOWN_TOKEN,UNKNOWN_TOKEN

x: These are the predictions made 
brown_bread,specialty_cheese,frankfurter,brown_bread,brown_bread,cocoa_drinks,canned_vegetables,baby_food,cleaner,brown_bread

y These are the actual present in dataset:
domestic_eggs,rolls/buns,brown_bread,UNKNOWN_TOKEN,UNKNOWN_TOKEN,UNKNOWN_TOKEN,UNKNOWN_TOKEN,UNKNOWN_TOKEN,UNKNOWN_TOKEN,UNKNOWN_TOKEN

x: These are the predictions made 
misc._beverages,brown_bread,berries,mustard,brown_bread,cookware,abrasive_cleaner,fish,abrasive_cleaner,liqueur

y These are the actual present in dataset:
rolls/buns,coffee,newspapers,UNKNOWN_TOKEN,UNKNOWN_TOKEN,UNKNOWN_TOKEN,UNKNOWN_TOKEN,UNKNOWN_TOKEN,UNKNOWN_TOKEN,UNKNOWN_TOKEN